In [1]:
import sys
import nltk
import torch.nn.functional as F
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
sys.path.insert(0, '..')
import pickle
from sklearn.preprocessing import LabelEncoder
from torch.autograd import grad as torch_grad
from sklearn.preprocessing import OneHotEncoder



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import torch


print(torch.__version__)
print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = "cpu"

2.0.1
GPU Available: False


In [3]:
batch_size = 64
word_embedding_dim = 50
output_shape = (1, 105, 8)
torch.manual_seed(1)
np.random.seed(1)

In [4]:

# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [5]:
def create_word_label_embeddings(Word_Labels_List):
    tokenized_words = []
    for i in range(len(Word_Labels_List)):
        tokenized_words.append([Word_Labels_List[i]])
    model = Word2Vec(sentences=tokenized_words, vector_size=word_embedding_dim, window=5, min_count=1, workers=4)
    word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}
    print("Number of word embeddings:", len(word_embeddings))
    #word, embedding = list(word_embeddings.items())[10]
    #print(f"Word: {word}, Embedding: {embedding}")

    Embedded_Word_labels = []
    for word in EEG_word_level_labels:
        Embedded_Word_labels.append(word_embeddings[word])

    return Embedded_Word_labels, word_embeddings

def create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels):
    EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.std(EEG_word_level_embeddings)

    float_tensor = torch.tensor(EEG_word_level_embeddings_normalize, dtype=torch.float)
    float_tensor = float_tensor.unsqueeze(1)

    #print(EEG_word_level_embeddings_normalize)
    # Calculate mean and standard deviation
    print(torch.isnan(float_tensor).any())

    train_data = []
    for i in range(len(float_tensor)):
       train_data.append([float_tensor[i], Embedded_Word_labels[i]])
    trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)
    return trainloader

In [76]:
#Embedded_Word_labels, word_embeddings = create_word_label_embeddings(EEG_word_level_labels)
encoder = LabelEncoder()
Embedded_Word_labels = encoder.fit_transform(np.array(EEG_word_level_labels).reshape(-1, 1))

Embedded_Word_labels = torch.tensor(Embedded_Word_labels, dtype=torch.float)


C:\Users\gxb18167\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [238]:
trainloader = create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels)

tensor(False)


In [239]:
z_size = 100
image_size = (105, 8)
n_classes = 5860
n_filters = 32

In [240]:
class Generator(nn.Module):
    def __init__(self, noise_dim, word_embedding_dim):
        super(Generator, self).__init__()

        self.noise_dim = noise_dim
        self.label_emb = nn.Embedding(n_classes, 100)

        # Define the layers of your generator
        self.fc_noise = nn.Linear(noise_dim, 105 * 8)
        self.fc_word_embedding = nn.Linear(100, 105 * 8)
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)



    def forward(self, noise, word_embedding):
        # Process noise
        noise = self.fc_noise(noise)
        noise = noise.view(noise.size(0), 1, 105, 8)

        # Process word embedding
        word_embedding = self.label_emb(word_embedding)

        word_embedding = self.fc_word_embedding(word_embedding)

        word_embedding = word_embedding.view(word_embedding.size(0), 1, 105, 8)

        # Concatenate noise and word embedding
        combined_input = torch.cat([noise, word_embedding], dim=1)

        # Upsample and generate the output
        z = self.conv1(combined_input)
        z = self.bn1(z)
        z = self.relu(z)
        z = self.conv2(z)

        return z

class DiscriminatorWGAN(nn.Module):
    def __init__(self, n_filters):
        super(DiscriminatorWGAN, self).__init__()

        self.network = nn.Sequential(
            nn.Conv2d(1, n_filters, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters, n_filters*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(n_filters * 2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters*2, n_filters*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(n_filters*4),
            nn.LeakyReLU(0.2),

            nn.Flatten(),  # Flatten spatial dimensions

            # Fully connected layer to reduce to a single value per sample

        )

        self.adv_layer = nn.Sequential(
            nn.Linear(n_filters*4 * (105 // 8) * (8 // 8), 1),
            nn.Sigmoid()
        )
        self.aux_layer = nn.Sequential(
            nn.Linear(n_filters*4 * (105 // 8) * (8 // 8), 1),
            nn.Softmax()
        )

    def forward(self, input):
        #print("combined_input:", combined_input.shape)

        output = self.network(input)
        valid = self.adv_layer(output)
        label = self.aux_layer(output)


        return valid, label


In [241]:
gen_model = Generator(z_size, 1).to(device)
disc_model = DiscriminatorWGAN(n_filters).to(device)
adversarial_loss = nn.BCELoss()
auxiliary_loss = nn.CrossEntropyLoss()
g_optimizer = torch.optim.Adam(gen_model.parameters(), 0.00002)
d_optimizer = torch.optim.Adam(disc_model.parameters(), 0.00002)

In [242]:
def create_noise(batch_size, z_size, mode_z):
    if mode_z == 'uniform':
        input_z = torch.rand(batch_size, z_size)*2 - 1
    elif mode_z == 'normal':
        input_z = torch.randn(batch_size, z_size)
    return input_z

mode_z = 'uniform'
fixed_z = create_noise(batch_size, z_size, mode_z).to(device)

def create_samples(g_model, input_z, input_t):
    g_output = g_model(input_z, input_t)
    images = torch.reshape(g_output, (batch_size, *image_size))
    return (images+1)/2.0

In [243]:

## Train the discriminator
def d_train(x, T):

    #init gradient
    disc_model.zero_grad()

    # Train discriminator with a real batch
    batch_size = x.size(0)
    x = x.to(device)
    d_proba_real, real_aux = disc_model(x)

    #Calculate loss (real images)
    d_labels_real = torch.ones(batch_size, 1, device=device)
    #d_loss_real = adversarial_loss(d_proba_real, d_labels_real)

    real_aux = real_aux.squeeze(dim=1)
    d_real_loss = (adversarial_loss(d_proba_real, d_labels_real) + auxiliary_loss(real_aux, T)) / 2

    #building Z
    input_z = create_noise(batch_size, z_size, mode_z).to(device)

    #building fake T
    gen_labels = torch.randint(0, n_classes, (batch_size,))

    # Train discriminator on a fake batch
    g_output = gen_model(input_z, gen_labels)

    #Discriminating fake images
    d_proba_fake, fake_aux = disc_model(g_output)

    #Calculate loss (fake images)
    d_labels_fake = torch.zeros(batch_size, 1, device=device)

    gen_labels = gen_labels.type(torch.float32).to(device)
    fake_aux = fake_aux.squeeze(dim=1)

    d_fake_loss = (adversarial_loss(d_proba_fake, d_labels_fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

    # gradient backprop & optimize ONLY D's parameters
    d_loss = (d_real_loss + d_fake_loss) / 2
    d_loss.backward()
    d_optimizer.step()

    return d_loss.data.item(), d_proba_real.detach(), d_proba_fake.detach()

In [244]:
## Train the generator
def g_train(x):

    #Init gradient
    gen_model.zero_grad()

    #Building Z
    batch_size = x.size(0)
    input_z = create_noise(batch_size, z_size, mode_z).to(device)

    #building real labels
    g_labels_real = torch.ones((batch_size, 1), device=device)

    #generate fake text embeddings
    gen_labels = torch.randint(0, n_classes, (batch_size,))

    g_output = gen_model(input_z, gen_labels)

    #in this case, d_proba_fake is their validity
    d_proba_fake, pred_label = disc_model(g_output)

    pred_label = pred_label.squeeze(dim=1)

    gen_labels = gen_labels.type(torch.float32).to(device)
    g_loss = 0.5 * (adversarial_loss(d_proba_fake, g_labels_real) + auxiliary_loss(pred_label, gen_labels))

    # gradient backprop & optimize ONLY G's parameters
    g_loss.backward()
    g_optimizer.step()

    return g_loss.data.item()

In [246]:
mode_z = 'uniform'
epoch_samples = []
num_epochs = 100
torch.manual_seed(1)
save_interval = 5
checkpoint_path = 'Textual_DCGAN_model_checkpoint_epoch_{}.pth'
final_model_path = 'Textual_DCGAN_model_final.pth'

for epoch in range(1, num_epochs+1):
    gen_model.train()
    fixed_z = create_noise(batch_size, z_size, mode_z).to(device)
    d_losses, g_losses = [], []
    for i, (x, t) in enumerate(trainloader):
        g_losses.append(g_train(x))
        d_loss, d_proba_real, d_proba_fake = d_train(x, t)
        print("D Loss:", d_loss)
        d_losses.append(d_loss)


    print(f'Epoch {epoch:03d} | D Loss >>'
          f' {torch.FloatTensor(d_losses).mean():.4f}')
    print(f'Epoch {epoch:03d} | G Loss >>'
          f' {torch.FloatTensor(g_losses).mean():.4f}')

    if epoch % save_interval == 0:
        torch.save({
            'epoch': epoch,
            'gen_model_state_dict': gen_model.state_dict(),
            'optimizer_state_dict': g_optimizer.state_dict(),
            'd_losses': d_losses,
            'g_losses': g_losses,
        }, checkpoint_path.format(epoch))

D Loss: 424373.5625
D Loss: 457704.9375
D Loss: 393035.3125
D Loss: 397463.5
D Loss: 412990.6875
D Loss: 392489.5
D Loss: 426612.0625
D Loss: 384024.0625
D Loss: 389095.8125
D Loss: 406857.375
D Loss: 403790.1875
D Loss: 453039.6875
D Loss: 416902.09375
D Loss: 437270.25
D Loss: 401462.28125
D Loss: 429782.1875
D Loss: 395222.9375
D Loss: 404133.3125
D Loss: 420572.3125
D Loss: 403210.0
D Loss: 399325.625
D Loss: 389183.15625
D Loss: 423793.375
D Loss: 422344.0
D Loss: 444082.46875
D Loss: 429303.875
D Loss: 396334.34375
D Loss: 427417.8125
D Loss: 399917.25
D Loss: 436652.625
D Loss: 386551.625
D Loss: 431187.875
D Loss: 388970.0
D Loss: 393410.65625
D Loss: 401817.875
D Loss: 406060.9375
D Loss: 386172.125
D Loss: 437405.375
D Loss: 407775.46875
D Loss: 415940.34375
D Loss: 435987.21875
D Loss: 404570.0
D Loss: 388072.75
D Loss: 426127.5625
D Loss: 424638.65625
D Loss: 366027.53125
D Loss: 415099.21875
D Loss: 424394.3125
D Loss: 405112.71875
D Loss: 430885.3125
D Loss: 395719.90625


KeyboardInterrupt: 